# Logging in Java Jupyter Notebooks

Use proper logging instead of `System.out.println()` with SLF4J and Log4j2

## 1. SLF4J with Logback (Recommended)

SLF4J is the industry standard logging facade with Logback implementation

In [1]:
// Add SLF4J API and Logback implementation
%maven org.slf4j:slf4j-api:2.0.12
%maven ch.qos.logback:logback-classic:1.5.3

In [2]:
import org.slf4j.Logger;
import org.slf4j.LoggerFactory;

// Create a logger
Logger log = LoggerFactory.getLogger("MyNotebook");

// Use different log levels
log.trace("This is a TRACE message");
log.debug("This is a DEBUG message");
log.info("This is an INFO message");
log.warn("This is a WARN message");
log.error("This is an ERROR message");

09:20:38.550 [IJava-executor-0] DEBUG MyNotebook -- This is a DEBUG message
09:20:38.564 [IJava-executor-0] INFO MyNotebook -- This is an INFO message
09:20:38.587 [IJava-executor-0] WARN MyNotebook -- This is a WARN message
09:20:38.606 [IJava-executor-0] ERROR MyNotebook -- This is an ERROR message


In [3]:
// Parameterized logging (efficient - no string concatenation)
String username = "Alice";
int age = 30;

log.info("User {} logged in at age {}", username, age);
log.warn("Failed login attempt for user: {}", username);

09:20:42.339 [IJava-executor-0] INFO MyNotebook -- User Alice logged in at age 30
09:20:42.348 [IJava-executor-0] WARN MyNotebook -- Failed login attempt for user: Alice


In [4]:
// Logging with exceptions
try {
    int result = 10 / 0;
} catch (Exception e) {
    log.error("Error occurred during calculation", e);
}

09:20:53.346 [IJava-executor-0] ERROR MyNotebook -- Error occurred during calculation
java.lang.ArithmeticException: / by zero
	at REPL.$JShell$30.do_it$($JShell$30.java:18)
	at java.base/jdk.internal.reflect.DirectMethodHandleAccessor.invoke(DirectMethodHandleAccessor.java:103)
	at java.base/java.lang.reflect.Method.invoke(Method.java:580)
	at io.github.spencerpark.ijava.execution.IJavaExecutionControl.lambda$execute$1(IJavaExecutionControl.java:95)
	at java.base/java.util.concurrent.FutureTask.run(FutureTask.java:317)
	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1144)
	at java.base/java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:642)
	at java.base/java.lang.Thread.run(Thread.java:1583)


## 2. Configure Logback Programmatically

In [ ]:
import ch.qos.logback.classic.Level;
import ch.qos.logback.classic.Logger;
import ch.qos.logback.classic.LoggerContext;
import ch.qos.logback.classic.encoder.PatternLayoutEncoder;
import ch.qos.logback.core.ConsoleAppender;

// Get logger context
LoggerContext loggerContext = (LoggerContext) LoggerFactory.getILoggerFactory();

// Create console appender
ConsoleAppender consoleAppender = new ConsoleAppender();
consoleAppender.setContext(loggerContext);
consoleAppender.setName("console");

// Create pattern encoder
PatternLayoutEncoder encoder = new PatternLayoutEncoder();
encoder.setContext(loggerContext);
encoder.setPattern("%d{HH:mm:ss.SSS} [%thread] %-5level %logger{36} - %msg%n");
encoder.start();

consoleAppender.setEncoder(encoder);
consoleAppender.start();

// Get root logger and configure
Logger rootLogger = loggerContext.getLogger(Logger.ROOT_LOGGER_NAME);
rootLogger.setLevel(Level.INFO);
rootLogger.addAppender(consoleAppender);

System.out.println("✅ Logback configured!");

In [ ]:
// Test the configured logger
Logger myLog = (Logger) LoggerFactory.getLogger("TestLogger");

myLog.debug("This won't show (DEBUG < INFO)");
myLog.info("Application started");
myLog.warn("Low memory warning");
myLog.error("Database connection failed");

## 3. Log4j2 (Alternative)

Log4j2 is a high-performance logging framework

In [ ]:
// Add Log4j2 dependencies
%maven org.apache.logging.log4j:log4j-api:2.23.0
%maven org.apache.logging.log4j:log4j-core:2.23.0

In [ ]:
import org.apache.logging.log4j.LogManager;
import org.apache.logging.log4j.Logger;

// Create Log4j2 logger
Logger logger = LogManager.getLogger("MyLog4j2Logger");

// Use different log levels
logger.trace("Trace message");
logger.debug("Debug message");
logger.info("Info message");
logger.warn("Warning message");
logger.error("Error message");
logger.fatal("Fatal message");

In [ ]:
// Parameterized logging with Log4j2
String user = "Bob";
int attempts = 3;

logger.info("User {} logged in after {} attempts", user, attempts);
logger.error("Failed to process order for user: {}", user);

## 4. Create a Simple Logger Wrapper

In [ ]:
import org.slf4j.Logger;
import org.slf4j.LoggerFactory;

// Simple logging utility class
class Log {
    private static final Logger logger = LoggerFactory.getLogger("Notebook");
    
    public static void debug(String message, Object... args) {
        logger.debug(message, args);
    }
    
    public static void info(String message, Object... args) {
        logger.info(message, args);
    }
    
    public static void warn(String message, Object... args) {
        logger.warn(message, args);
    }
    
    public static void error(String message, Object... args) {
        logger.error(message, args);
    }
    
    public static void error(String message, Throwable throwable) {
        logger.error(message, throwable);
    }
}

System.out.println("✅ Log utility class created!");

In [ ]:
// Now use the simple Log class
Log.info("Application started");
Log.warn("Cache is almost full: {}%", 85);
Log.error("Failed to connect to database");

try {
    int x = 1 / 0;
} catch (Exception e) {
    Log.error("Division error occurred", e);
}

## 5. Real-World Example: Data Processing with Logging

In [ ]:
import java.util.*;
import java.util.stream.*;

class DataProcessor {
    private static final Logger log = LoggerFactory.getLogger(DataProcessor.class);
    
    public static List<Integer> processData(List<Integer> data) {
        log.info("Starting data processing with {} items", data.size());
        
        var filtered = data.stream()
            .filter(n -> {
                if (n < 0) {
                    log.warn("Negative value found: {}", n);
                    return false;
                }
                return true;
            })
            .map(n -> n * 2)
            .collect(Collectors.toList());
        
        log.info("Processing complete. Output size: {}", filtered.size());
        return filtered;
    }
}

// Test the processor
List<Integer> input = Arrays.asList(1, -2, 3, 4, -5, 6);
List<Integer> output = DataProcessor.processData(input);

Log.info("Final result: {}", output);

## 6. Comparison: System.out vs Logging

### ❌ System.out.println() Problems:
- No log levels (can't filter by severity)
- No timestamps
- No thread information
- No easy way to disable in production
- Poor performance
- Can't redirect to files easily

### ✅ Logging Benefits:
- Multiple log levels (TRACE, DEBUG, INFO, WARN, ERROR)
- Automatic timestamps and thread info
- Can be configured at runtime
- Better performance (lazy evaluation)
- Can output to console, files, databases, etc.
- Industry standard for production code

## 7. Log Levels Explained

| Level | When to Use | Example |
|-------|-------------|----------|
| **TRACE** | Very detailed diagnostic info | `log.trace("Entering method calculateTotal()")` |
| **DEBUG** | Debugging information | `log.debug("User query returned {} results", count)` |
| **INFO** | General informational messages | `log.info("Application started successfully")` |
| **WARN** | Warning messages (recoverable) | `log.warn("Database connection slow: {}ms", time)` |
| **ERROR** | Error messages (app still running) | `log.error("Failed to process order", exception)` |
| **FATAL** | Critical errors (app may crash) | `log.fatal("Database unreachable, shutting down")` |

## Best Practices

1. **Use parameterized logging** instead of string concatenation:
   ```java
   // ✅ Good
   log.info("User {} logged in", username);
   
   // ❌ Bad
   log.info("User " + username + " logged in");
   ```

2. **Choose the right log level**:
   - Production: INFO and above
   - Development: DEBUG and above
   - Troubleshooting: TRACE

3. **Always log exceptions with context**:
   ```java
   catch (Exception e) {
       log.error("Failed to process order {}", orderId, e);
   }
   ```

4. **Use meaningful logger names**:
   ```java
   Logger log = LoggerFactory.getLogger(MyClass.class);
   ```

5. **Don't log sensitive information** (passwords, credit cards, etc.)